In [1]:
import pandas as pd
import requests
import os
import re

pd.options.display.max_columns = 999
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
if not os.path.exists('./nfl_data/weekly/cbs_weekly'):
    os.mkdir('./nfl_data/weekly/cbs_weekly')

#### Scrapes CBS Weekly for all positions & all weeks
Saves files to `cbs_weekly` directory

In [3]:
positions = ['QB', 'WR', 'RB', 'TE', 'K']
weeks = [19, 20, 21, 23] # 22 was pro bowl

file_list = []

for position in positions:
    for week in weeks:
        url = f'https://www.cbssports.com/nfl/stats/leaders/live/{position}/{week}/'
        print(url)
        res = requests.get(url)
        print(res)
        output = open(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls', 'wb')
        output.write(res.content)
        file_list.append(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        output.close()
        data = pd.read_html(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        df = data[0]
        # df = df.droplevel(0, axis=1)
        print((position, week), df.shape)

https://www.cbssports.com/nfl/stats/leaders/live/QB/19/
<Response [200]>
('QB', 19) (13, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/20/
<Response [200]>
('QB', 20) (8, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/21/
<Response [200]>
('QB', 21) (3, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/23/
<Response [200]>
('QB', 23) (2, 11)
https://www.cbssports.com/nfl/stats/leaders/live/WR/19/
<Response [200]>
('WR', 19) (56, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/20/
<Response [200]>
('WR', 20) (39, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/21/
<Response [200]>
('WR', 21) (14, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/23/
<Response [200]>
('WR', 23) (10, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/19/
<Response [200]>
('RB', 19) (35, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/20/
<Response [200]>
('RB', 20) (19, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/21/
<Response [200]>


In [4]:
file_list

['./nfl_data/weekly/cbs_weekly/week-19_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-20_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-21_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-23_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-19_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-20_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-21_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-23_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-19_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-20_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-21_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-23_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-19_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-20_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-21_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-23_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-19_K.xls',
 './nfl_data/weekly/cbs_weekly/week-20_K.xls',
 './nfl_data/weekly/cbs_weekly/week-21_K.xls',
 './nfl_data/weekly/cbs_weekly/week-23_K.xls']

In [5]:
# get df started with the 1st file './nfl_data/week-19_WR.xls'.  This way we have something to merge to.

df_wr = pd.read_html('./nfl_data/weekly/cbs_weekly/week-19_WR.xls')
df_wr = pd.DataFrame(df_wr[0])
df_wr = df_wr.droplevel(0, axis=1)
df_wr['Week'] = re.search(r'(?<=\-)\s*(..)', file_list[0])[0]
df_wr['Pos'] = [i.split()[-1] for i in df_wr['Player  Player on team']]
df_wr.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
df_wr['Player'] = df_wr['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
df_wr

,Player,Game,FPTS Fantasy Points,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
0,Kendrick Bourne,BUF 47 - NE 17,20,7,77,8,2,1,14,0,19,WR
1,Mike Evans,TB 31 - PHI 15,17,9,117,10,1,—,—,—,19,WR
2,Deebo Samuel,SF 23 - DAL 17,16,3,38,3,0,10,72,1,19,WR
3,Byron Pringle,KC 42 - PIT 21,15,5,37,7,2,—,—,—,19,WR
4,Ja'Marr Chase,CIN 26 - LV 19,13,9,116,12,0,3,23,0,19,WR
5,Amari Cooper,SF 23 - DAL 17,12,6,64,10,1,—,—,—,19,WR
6,WR Odell,LAR 34 - ARI 11,12,4,54,4,1,—,—,—,19,WR
7,Zay Jones,CIN 26 - LV 19,12,5,61,8,1,—,—,—,19,WR
8,Cooper Kupp,LAR 34 - ARI 11,12,5,61,7,1,—,—,—,19,WR
9,Tyreek Hill,KC 42 - PIT 21,11,5,57,5,1,—,—,—,19,WR


In [6]:
df_wr.shape

(56, 12)

In [7]:
for file in file_list:
    if 'WR' in file:
        print(file)
        data = pd.read_html(file)
        df = data[0]
        df = df.droplevel(0, axis=1)
        df['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
        df['Pos'] = [i.split()[-1] for i in df['Player  Player on team']]
        df.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
        df['Player'] = df['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
        dfs = [df_wr, df]
        df_wr = pd.concat([df_wr.squeeze() for df_wr in dfs], ignore_index=True)

print(f'The shape of the updated df is {df_wr.shape}')

./nfl_data/weekly/cbs_weekly/week-19_WR.xls
./nfl_data/weekly/cbs_weekly/week-20_WR.xls
./nfl_data/weekly/cbs_weekly/week-21_WR.xls
./nfl_data/weekly/cbs_weekly/week-23_WR.xls
The shape of the updated df is (175, 12)


In [8]:
df_wr.sort_values(by=['Player', 'Week'])

,Player,Game,FPTS Fantasy Points,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
115,A.J. Brown,CIN 19 - TEN 16,25,5,142,9,1,—,—,—,20,WR
42,A.J. Green,LAR 34 - ARI 11,—,0,0,3,0,—,—,—,19,WR
98,A.J. Green,LAR 34 - ARI 11,—,0,0,3,0,—,—,—,19,WR
133,Allen Lazard,SF 13 - GB 10,1,1,6,1,0,—,—,—,20,WR
5,Amari Cooper,SF 23 - DAL 17,12,6,64,10,1,—,—,—,19,WR
61,Amari Cooper,SF 23 - DAL 17,12,6,64,10,1,—,—,—,19,WR
141,Amari Rodgers,SF 13 - GB 10,—,—,—,—,—,—,—,—,20,WR
55,Antoine Wesley,LAR 34 - ARI 11,—,0,0,2,0,—,—,—,19,WR
111,Antoine Wesley,LAR 34 - ARI 11,—,0,0,2,0,—,—,—,19,WR
149,Ben Skowronek,LAR 30 - TB 27,—,—,—,—,—,—,—,—,20,WR


In [9]:
df_wr.columns

Index(['Player', 'Game', 'FPTS  Fantasy Points', 'REC  Receptions',
       'YDS  Receiving Yards', 'TGT  Targets', 'TD  Receiving Touchdowns',
       'ATT  Rushing Attempts', 'YDS  Rushing Yards', 'TD  Rushing Touchdowns',
       'Week', 'Pos'],
      dtype='object')

In [10]:
df_wr = df_wr.replace('—', 0) # get rid of the dashes, impute with 0
df_wr.head()

,Player,Game,FPTS Fantasy Points,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
0,Kendrick Bourne,BUF 47 - NE 17,20,7,77,8,2,1,14,0,19,WR
1,Mike Evans,TB 31 - PHI 15,17,9,117,10,1,0,0,0,19,WR
2,Deebo Samuel,SF 23 - DAL 17,16,3,38,3,0,10,72,1,19,WR
3,Byron Pringle,KC 42 - PIT 21,15,5,37,7,2,0,0,0,19,WR
4,Ja'Marr Chase,CIN 26 - LV 19,13,9,116,12,0,3,23,0,19,WR


In [11]:
# Attempt to convert all columns to int, those that fail are stored in list
numcols_to_change = df_wr.columns
numcols_to_change2 = []
for col in numcols_to_change:
    try:
        df_wr[col] = df_wr[col].astype(int)
        print('success!')
    except:
        numcols_to_change2.append(col)
        print(f'need to clean column: {col}')

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos


In [13]:
max_week = df_wr['Week'].max()
df_wr.to_csv(f'./nfl_data/weekly/cbs_weekly/receiving_through_week-{max_week}.csv', index=0)